In [2]:
import pandas as pd
import numpy as np
import geopandas as gpd
import maup
from shapely.geometry import Point
import matplotlib.pyplot as plt

In [10]:
neighborhoods = gpd.read_file('Boston_Neighborhoods.shp')
precincts = gpd.read_file('Precincts.shp')


In [11]:
precincts = precincts[['WARD_PRECI', 'geometry']]
precincts = precincts.set_crs(epsg=26986)
precincts.head()

,WARD_PRECI,geometry
0,0113,"POLYGON ((787723.729 2965812.641, 787659.076 2..."
1,0102,"POLYGON ((782820.000 2960295.100, 783161.200 2..."
2,0105,"POLYGON ((783942.500 2960316.200, 783754.100 2..."
3,0203,"POLYGON ((775776.888 2961852.579, 775644.100 2..."
4,0205,"POLYGON ((774156.600 2964357.700, 774555.300 2..."


In [12]:
precincts[precincts['WARD_PRECI'] == '0113']['geometry'].area

0    7.562361e+06
dtype: float64

In [13]:
neighborhoods = neighborhoods[['Name', 'geometry']]
neighborhoods = neighborhoods.set_crs(epsg=26986)
neighborhoods.head()

,Name,geometry
0,Roslindale,"MULTIPOLYGON (((757409.123 2924367.749, 757457..."
1,Jamaica Plain,"POLYGON ((762983.826 2944104.016, 763014.250 2..."
2,Mission Hill,"POLYGON ((766903.580 2947645.876, 766280.013 2..."
3,Longwood,"POLYGON ((764826.911 2947985.998, 764607.180 2..."
4,Bay Village,"POLYGON ((773315.750 2952419.751, 773111.500 2..."


In [14]:
join = gpd.sjoin(neighborhoods, precincts)

In [15]:
join

,Name,geometry,index_right,WARD_PRECI
0,Roslindale,"MULTIPOLYGON (((757409.123 2924367.749, 757457...",148,1814
19,Hyde Park,"POLYGON ((757421.375 2924404.428, 757451.934 2...",148,1814
0,Roslindale,"MULTIPOLYGON (((757409.123 2924367.749, 757457...",149,1822
19,Hyde Park,"POLYGON ((757421.375 2924404.428, 757451.934 2...",149,1822
0,Roslindale,"MULTIPOLYGON (((757409.123 2924367.749, 757457...",146,1811
...,...,...,...,...
23,South Boston,"POLYGON ((774838.800 2950258.201, 774877.100 2...",25,0605
24,Allston,"POLYGON ((754791.024 2951324.323, 753961.500 2...",83,2105
24,Allston,"POLYGON ((754791.024 2951324.323, 753961.500 2...",193,2103
24,Allston,"POLYGON ((754791.024 2951324.323, 753961.500 2...",82,2104


In [23]:
merge = join[['WARD_PRECI', 'Name']]
merge = merge.reset_index(drop=True)
merge = merge.rename(columns={'Name': 'NEIGHBORHOOD'})
merge.to_csv('WardPrecinct_Matched_To_Neighborhood.csv')
merge

,WARD_PRECI,NEIGHBORHOOD
0,1814,Roslindale
1,1814,Hyde Park
2,1822,Roslindale
3,1822,Hyde Park
4,1811,Roslindale
...,...,...
399,0605,South Boston
400,2105,Allston
401,2103,Allston
402,2104,Allston


In [17]:
merge[merge['NEIGHBORHOOD'] == 'South Boston Waterfront']

,WARD_PRECI,NEIGHBORHOOD
385,0604,South Boston Waterfront
387,0601,South Boston Waterfront


In [18]:
len(precincts['WARD_PRECI'].unique())

255

In [19]:
len(merge['WARD_PRECI'].unique())

255

In [25]:
cols = precincts['WARD_PRECI'].unique().tolist()

props = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])), columns = cols)
props.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(merge)):
    neighb = merge['NEIGHBORHOOD'].loc[i]
    prct = merge['WARD_PRECI'].loc[i]
    filt = props['NEIGHBORHOOD'] == neighb
    intersection = gpd.overlay(precincts[precincts['WARD_PRECI'] == prct], neighborhoods[neighborhoods['Name'] == neighb], 
                         how='intersection')
    shape1 = intersection['geometry'].loc[0]
    shape2 = neighborhoods[neighborhoods['Name'] == neighb]['geometry']
    indexes = props.index
    index = indexes[filt]
    props.loc[index, prct] = shape1.area / shape2.area

In [64]:
props

,NEIGHBORHOOD,0113,0102,0105,0203,0205,0204,0206,0303,0304,...,1812,1823,2003,1816,1817,1818,0502A,0115,0104,2002
0,Roslindale,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.065924
1,Jamaica Plain,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Mission Hill,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Longwood,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Bay Village,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,Leather District,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
6,Chinatown,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,North End,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.211189,0.121440,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Roxbury,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,South End,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [24]:
props.to_csv('Precinct_Make_Up_Of_Neighborhoods.csv')

NameError: name 'props' is not defined

In [20]:
cols = neighborhoods['Name'].unique().tolist()

props2 = pd.DataFrame(0, index=np.arange(len(precincts['WARD_PRECI'])), columns = cols)
props2.insert(0, 'WARD_PRECI', precincts['WARD_PRECI'])

for i in range(len(merge)):
    neighb = merge['NEIGHBORHOOD'].loc[i]
    prct = merge['WARD_PRECI'].loc[i]
    filt = props2['WARD_PRECI'] == prct
    intersection = gpd.overlay(precincts[precincts['WARD_PRECI'] == prct], neighborhoods[neighborhoods['Name'] == neighb], 
                         how='intersection')
    shape1 = intersection['geometry'].loc[0]
    shape2 = precincts[precincts['WARD_PRECI'] == prct]['geometry']
    indexes = props2.index
    index = indexes[filt]
    props2.loc[index, neighb] = shape1.area / shape2.area

In [67]:
props2

,WARD_PRECI,Roslindale,Jamaica Plain,Mission Hill,Longwood,Bay Village,Leather District,Chinatown,North End,Roxbury,...,Fenway,Brighton,West Roxbury,Hyde Park,Mattapan,Dorchester,South Boston Waterfront,South Boston,Allston,Harbor Islands
0,0113,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
1,0102,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
2,0105,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
3,0203,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
4,0205,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,1818,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.000000
251,0502A,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000
252,0115,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.993271
253,0104,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000


In [68]:
props2.to_csv('Neighborhood_Make_Up_Of_Precincts.csv')

In [108]:
df11 = pd.read_csv('City Council/2011_November Municipal (General) City Council At Large Results.csv')
temp = df11['Ward/Precinct']
df11 = df11[['Year','VOTES CAST', 'BLANKS', 'BALLOTS CAST', 'Registered Voters']]
df11.insert(0, 'WARD_PRECINCT_CODE', temp)
df11 = df11[:254]
df11.loc[253, 'Year'] = 2011.0 # MISSING DATA
df11

,WARD_PRECINCT_CODE,Year,VOTES CAST,BLANKS,BALLOTS CAST,Registered Voters
0,101,2011.0,583,249,208,1114
1,102,2011.0,282,186,117,805
2,103,2011.0,695,385,270,1726
3,104,2011.0,217,167,96,548
4,105,2011.0,341,195,134,955
...,...,...,...,...,...,...
249,2209,2011.0,386,118,126,1302
250,2210,2011.0,645,179,206,1504
251,2211,2011.0,379,133,128,1005
252,2212,2011.0,427,233,165,1064


In [107]:
df13 = pd.read_csv('City Council/2013_ November Municipal (General) City Council At Large Results.csv')
temp = df13['WP']
df13 = df13[['Year','VOTES CAST', 'BLANKS', 'BALLOTS CAST', 'Registered Voters']]
df13.insert(0, 'WARD_PRECINCT_CODE', temp)
df13 = df13[:254]
df13.loc[253, 'Year'] = 2013.0 # MISSING DATA
df13

,WARD_PRECINCT_CODE,Year,VOTES CAST,BLANKS,BALLOTS CAST,Registered Voters
0,101,2013.0,1348,1224,643,1236
1,102,2013.0,808,920,432,952
2,103,2013.0,1801,1655,864,1965
3,104,2013.0,499,657,289,655
4,105,2013.0,963,865,457,1074
...,...,...,...,...,...,...
249,2209,2013.0,798,566,341,1283
250,2210,2013.0,1420,756,544,1532
251,2211,2013.0,883,561,361,1065
252,2212,2013.0,895,585,370,1144


In [106]:
df15 = pd.read_csv('City Council/2015_November Municipal (General) City Council At Large Results.csv')
temp = df15['WP']
df15 = df15[['Year','VOTES CAST', 'BLANKS', 'BALLOTS CAST', 'Registered Voters']]
df15.insert(0, 'WARD_PRECINCT_CODE', temp)
df15 = df15[:254]
df15.loc[253, 'Year'] = 2015.0 # MISSING DATA
df15

,WARD_PRECINCT_CODE,Year,VOTES CAST,BLANKS,BALLOTS CAST,Registered Voters
0,101,2015.0,479,217,174,1286
1,102,2015.0,249,179,107,949
2,103,2015.0,731,385,279,2074
3,104,2015.0,120,120,60,669
4,105,2015.0,229,119,87,1119
...,...,...,...,...,...,...
249,2209,2015.0,250,122,93,1236
250,2210,2015.0,491,189,170,1500
251,2211,2015.0,319,141,115,1057
252,2212,2015.0,368,184,138,1166


In [105]:
df17 = pd.read_csv('City Council/2017_November Municipal (General) City Council At Large_Results.csv')
temp = df17['Ward_Precinct']
df17 = df17[['Year','VOTES CAST', 'BLANKS', 'BALLOTS CAST', 'Registered Voters']]
df17.insert(0, 'WARD_PRECINCT_CODE', temp)
df17 = df17[:254]
df17.loc[253, 'Year'] = 2017.0 # MISSING DATA
df17

,WARD_PRECINCT_CODE,Year,VOTES CAST,BLANKS,BALLOTS CAST,Registered Voters
0,101.0,2017.0,1204,776,495,1397.0
1,102.0,2017.0,601,591,298,1035.0
2,103.0,2017.0,1565,1279,711,2379.0
3,104.0,2017.0,467,453,230,798.0
4,105.0,2017.0,806,614,355,1255.0
...,...,...,...,...,...,...
249,2209.0,2017.0,759,377,284,1328.0
250,2210.0,2017.0,1080,484,391,1455.0
251,2211.0,2017.0,648,412,265,1041.0
252,2212.0,2017.0,739,397,284,1200.0


In [113]:
wards = pd.concat([df11, df13, df15, df17])
wards = wards.reset_index(drop=True)
wards['Voter Turnout'] = pd.Series([0 for x in range(len(wards))])
for i in range(len(wards)):
    wards['Voter Turnout'].loc[i] = round(wards['VOTES CAST'].loc[i]/wards['Registered Voters'].loc[i],2)

wards.to_csv('2011-2017_Boston_Ward_Precinct_Voter_Data.csv')

df19 = pd.read_csv('City Council/2019_November Municipal (General) City Council At Large Results.csv')
temp = df19['WP']
df19 = df19[['Ballots', 'Voters', 'Total']]
df19.insert(0, 'WARD_PRECINCT_CODE', temp)
df19.columns = ['WARD_PRECINCT_CODE', 'BALLOTS CAST','Registered Voters', 'Total']
df19 = df19.drop([0])
df19 = df19.reset_index(drop=True)
df19.loc[253] = [2213.0, 0, 0, 0] # MISSING DATA
df19

df19[df19['WARD_PRECINCT_CODE'] == 604.0]

df19[df19['WARD_PRECINCT_CODE'] == 601.0]

In [90]:
adjusted11 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=['YEAR', 'VOTES_CAST', 'BLANKS', 'BALLOTS_CAST', 'REGISTERED_VOTERS'])
adjusted11.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(adjusted11)):
    neighb = adjusted11['NEIGHBORHOOD'].loc[i]
    temp = props2[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            adjusted11['YEAR'].loc[i] = 2011
            if temp['WARD_PRECI'].loc[n] == '0502A':
                adjusted11['VOTES_CAST'].loc[i] += round(int(df11[df11['WARD_PRECINCT_CODE'] == 502]['VOTES CAST']) * dec)
                adjusted11['BLANKS'].loc[i] += round(int(df11[df11['WARD_PRECINCT_CODE'] == 502]['BLANKS']) * dec)
                adjusted11['BALLOTS_CAST'].loc[i] += round(int(df11[df11['WARD_PRECINCT_CODE'] == 502]['BALLOTS CAST']) * dec)
                adjusted11['REGISTERED_VOTERS'].loc[i] += round(int(df11[df11['WARD_PRECINCT_CODE'] == 502]['Registered Voters']) * dec)
            else:
                filt = df11['WARD_PRECINCT_CODE'] == int(temp['WARD_PRECI'].loc[n])
                adjusted11['VOTES_CAST'].loc[i] += round(int(df11[filt]['VOTES CAST']) * dec)
                adjusted11['BLANKS'].loc[i] += round(int(df11[filt]['BLANKS']) * dec)
                adjusted11['BALLOTS_CAST'].loc[i] += round(int(df11[filt]['BALLOTS CAST']) * dec)
                adjusted11['REGISTERED_VOTERS'].loc[i] += round(int(df11[filt]['Registered Voters']) * dec)

/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [92]:
adjusted13 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=['YEAR', 'VOTES_CAST', 'BLANKS', 'BALLOTS_CAST', 'REGISTERED_VOTERS'])
adjusted13.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(adjusted13)):
    neighb = adjusted13['NEIGHBORHOOD'].loc[i]
    temp = props2[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            adjusted13['YEAR'].loc[i] = 2013
            if temp['WARD_PRECI'].loc[n] == '0502A':
                adjusted13['VOTES_CAST'].loc[i] += round(int(df13[df13['WARD_PRECINCT_CODE'] == 502]['VOTES CAST']) * dec)
                adjusted13['BLANKS'].loc[i] += round(int(df13[df13['WARD_PRECINCT_CODE'] == 502]['BLANKS']) * dec)
                adjusted13['BALLOTS_CAST'].loc[i] += round(int(df13[df13['WARD_PRECINCT_CODE'] == 502]['BALLOTS CAST']) * dec)
                adjusted13['REGISTERED_VOTERS'].loc[i] += round(int(df13[df13['WARD_PRECINCT_CODE'] == 502]['Registered Voters']) * dec)
            else:
                filt = df13['WARD_PRECINCT_CODE'] == int(temp['WARD_PRECI'].loc[n])
                adjusted13['VOTES_CAST'].loc[i] += round(int(df13[filt]['VOTES CAST']) * dec)
                adjusted13['BLANKS'].loc[i] += round(int(df13[filt]['BLANKS']) * dec)
                adjusted13['BALLOTS_CAST'].loc[i] += round(int(df13[filt]['BALLOTS CAST']) * dec)
                adjusted13['REGISTERED_VOTERS'].loc[i] += round(int(df13[filt]['Registered Voters']) * dec)

/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [94]:
adjusted15 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=['YEAR', 'VOTES_CAST', 'BLANKS', 'BALLOTS_CAST', 'REGISTERED_VOTERS'])
adjusted15.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(adjusted15)):
    neighb = adjusted15['NEIGHBORHOOD'].loc[i]
    temp = props2[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            adjusted15['YEAR'].loc[i] = 2015
            if temp['WARD_PRECI'].loc[n] == '0502A':
                adjusted15['VOTES_CAST'].loc[i] += round(int(df15[df15['WARD_PRECINCT_CODE'] == 502]['VOTES CAST']) * dec)
                adjusted15['BLANKS'].loc[i] += round(int(df15[df15['WARD_PRECINCT_CODE'] == 502]['BLANKS']) * dec)
                adjusted15['BALLOTS_CAST'].loc[i] += round(int(df15[df15['WARD_PRECINCT_CODE'] == 502]['BALLOTS CAST']) * dec)
                adjusted15['REGISTERED_VOTERS'].loc[i] += round(int(df15[df15['WARD_PRECINCT_CODE'] == 502]['Registered Voters']) * dec)
            else:
                filt = df15['WARD_PRECINCT_CODE'] == int(temp['WARD_PRECI'].loc[n])
                adjusted15['VOTES_CAST'].loc[i] += round(int(df15[filt]['VOTES CAST']) * dec)
                adjusted15['BLANKS'].loc[i] += round(int(df15[filt]['BLANKS']) * dec)
                adjusted15['BALLOTS_CAST'].loc[i] += round(int(df15[filt]['BALLOTS CAST']) * dec)
                adjusted15['REGISTERED_VOTERS'].loc[i] += round(int(df15[filt]['Registered Voters']) * dec)

/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

In [96]:
adjusted17 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=['YEAR', 'VOTES_CAST', 'BLANKS', 'BALLOTS_CAST', 'REGISTERED_VOTERS'])
adjusted17.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(adjusted17)):
    neighb = adjusted17['NEIGHBORHOOD'].loc[i]
    temp = props2[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            adjusted17['YEAR'].loc[i] = 2017
            if temp['WARD_PRECI'].loc[n] == '0502A':
                adjusted17['VOTES_CAST'].loc[i] += round(int(df17[df17['WARD_PRECINCT_CODE'] == 502]['VOTES CAST']) * dec)
                adjusted17['BLANKS'].loc[i] += round(int(df17[df17['WARD_PRECINCT_CODE'] == 502]['BLANKS']) * dec)
                adjusted17['BALLOTS_CAST'].loc[i] += round(int(df17[df17['WARD_PRECINCT_CODE'] == 502]['BALLOTS CAST']) * dec)
                adjusted17['REGISTERED_VOTERS'].loc[i] += round(int(df17[df17['WARD_PRECINCT_CODE'] == 502]['Registered Voters']) * dec)
            else:
                filt = df17['WARD_PRECINCT_CODE'] == int(temp['WARD_PRECI'].loc[n])
                adjusted17['VOTES_CAST'].loc[i] += round(int(df17[filt]['VOTES CAST']) * dec)
                adjusted17['BLANKS'].loc[i] += round(int(df17[filt]['BLANKS']) * dec)
                adjusted17['BALLOTS_CAST'].loc[i] += round(int(df17[filt]['BALLOTS CAST']) * dec)
                adjusted17['REGISTERED_VOTERS'].loc[i] += round(int(df17[filt]['Registered Voters']) * dec)

/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)
/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/s

adjusted19 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=['YEAR', 'BALLOTS_CAST', 'REGISTERED_VOTERS', 'TOTAL'])
adjusted19.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(adjusted19)):
    neighb = adjusted19['NEIGHBORHOOD'].loc[i]
    temp = props2[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            adjusted19['YEAR'].loc[i] = 2019
            if temp['WARD_PRECI'].loc[n] == '0502A':
                adjusted19['BALLOTS_CAST'].loc[i] += int(df19[df19['WARD_PRECINCT_CODE'] == 502]['BALLOTS CAST']) * dec
                adjusted19['REGISTERED_VOTERS'].loc[i] += int(df19[df19['WARD_PRECINCT_CODE'] == 502]['Registered Voters']) * dec
                adjusted19['TOTAL'].loc[i] += int(df19[df19['WARD_PRECINCT_CODE'] == 502]['Total']) * dec
            else:
                filt = df19['WARD_PRECINCT_CODE'] == int(temp['WARD_PRECI'].loc[n])
                adjusted19['BALLOTS_CAST'].loc[i] += int(df19[filt]['BALLOTS CAST']) * dec
                adjusted19['REGISTERED_VOTERS'].loc[i] += int(df19[filt]['Registered Voters']) * dec
                adjusted19['TOTAL'].loc[i] += int(df19[filt]['Total']) * dec

adjusted19

In [97]:
finished = pd.concat([adjusted11, adjusted13, adjusted15, adjusted17])
finished = finished.reset_index(drop=True)
finished

,NEIGHBORHOOD,YEAR,VOTES_CAST,BLANKS,BALLOTS_CAST,REGISTERED_VOTERS
0,Roslindale,2011,10662,3741,3600,16695
1,Jamaica Plain,2011,16150,5542,5421,26678
2,Mission Hill,2011,2558,858,854,6506
3,Longwood,2011,201,67,66,851
4,Bay Village,2011,483,323,202,876
...,...,...,...,...,...,...
99,Dorchester,2017,46753,32822,19896,70924
100,South Boston Waterfront,2017,2677,1547,1056,5192
101,South Boston,2017,16214,12994,7302,26275
102,Allston,2017,5568,2493,2014,11802


In [99]:
finished['Voter Turnout'] = pd.Series([0 for x in range(len(finished))])
for i in range(len(finished)):
    finished['Voter Turnout'].loc[i] = round(finished['VOTES_CAST'].loc[i]/finished['REGISTERED_VOTERS'].loc[i],2)

finished

/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,NEIGHBORHOOD,YEAR,VOTES_CAST,BLANKS,BALLOTS_CAST,REGISTERED_VOTERS,Voter Turnout
0,Roslindale,2011,10662,3741,3600,16695,0.64
1,Jamaica Plain,2011,16150,5542,5421,26678,0.61
2,Mission Hill,2011,2558,858,854,6506,0.39
3,Longwood,2011,201,67,66,851,0.24
4,Bay Village,2011,483,323,202,876,0.55
...,...,...,...,...,...,...,...
99,Dorchester,2017,46753,32822,19896,70924,0.66
100,South Boston Waterfront,2017,2677,1547,1056,5192,0.52
101,South Boston,2017,16214,12994,7302,26275,0.62
102,Allston,2017,5568,2493,2014,11802,0.47


In [100]:
finished.to_csv('2011-2017_Estimated_Voter_Data_Per_Neighborhood.csv')

In [3]:
df1015 = pd.read_csv('Precinct_Demographics_Based_On_2006-11_ACS_Census_Data.csv')
df1519 = pd.read_csv('Precinct_Demographics_Based_On_2015-19_ACS_Census_Data.csv')
df1015 = df1015.drop('Unnamed: 0', axis=1)
df1519 = df1519.drop('Unnamed: 0', axis=1)

In [21]:
neigh1015 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=df1015.columns[1:])
neigh1015.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh1015)):
    neighb = neigh1015['NEIGHBORHOOD'].loc[i]
    temp = props2[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = df1015['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            neigh1015['TOTAL'].iloc[i] += df1015[filt]['TOTAL'] * dec
            neigh1015['WHITE'].iloc[i] += df1015[filt]['WHITE'] * dec
            neigh1015['AFRICAN_AMERICAN'].iloc[i] += df1015[filt]['AFRICAN_AMERICAN'] * dec
            neigh1015['NATIVE_AMERICAN'].iloc[i] += df1015[filt]['NATIVE_AMERICAN'] * dec
            neigh1015['ASIAN'].iloc[i] += df1015[filt]['ASIAN'] * dec
            neigh1015['OTHER'].iloc[i] += df1015[filt]['OTHER'] * dec

for c in neigh1015.columns:
    if c != 'NEIGHBORHOOD':
        neigh1015[c] = neigh1015[c].round().astype(int)

cols = list(neigh1015.columns)[2:]

for c in cols:
    for i in range(len(neigh1015)):
        neigh1015.loc[i, c] = round(neigh1015.loc[i, c]/neigh1015.loc[i, 'TOTAL'], 2)

neigh1015

/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,NEIGHBORHOOD,TOTAL,WHITE,AFRICAN_AMERICAN,NATIVE_AMERICAN,ASIAN,OTHER
0,Roslindale,27403,0.58,0.25,0.00,0.04,0.08
1,Jamaica Plain,39409,0.58,0.18,0.01,0.06,0.12
2,Mission Hill,13116,0.54,0.21,0.00,0.13,0.09
3,Longwood,5901,0.64,0.10,0.00,0.15,0.07
4,Bay Village,1334,0.63,0.03,0.00,0.30,0.02
5,Leather District,489,0.41,0.04,0.01,0.48,0.03
6,Chinatown,2487,0.43,0.04,0.01,0.47,0.03
7,North End,6385,0.93,0.03,0.00,0.03,0.01
8,Roxbury,45039,0.18,0.59,0.00,0.03,0.15
9,South End,23004,0.59,0.12,0.00,0.18,0.09


In [22]:
neigh1015.to_csv('Boston_Neighborhood_Demographics_Based_On_ACS_5Year_Estimates_2006-11.csv')

In [88]:
neigh1519 = pd.DataFrame(0, index=np.arange(len(neighborhoods['Name'])),
                        columns=df1519.columns[1:])
neigh1519.insert(0, 'NEIGHBORHOOD', neighborhoods['Name'])

for i in range(len(neigh1519)):
    neighb = neigh1519['NEIGHBORHOOD'].loc[i]
    temp = props2[['WARD_PRECI', neighb]]
    for n in range(len(temp)):
        if temp[neighb].loc[n] != 0:
            dec = temp[neighb].loc[n]
            filt = df1519['WARD_PRECI'] == temp['WARD_PRECI'].loc[n]
            neigh1519['TOTAL'].iloc[i] += df1519[filt]['TOTAL'] * dec
            neigh1519['WHITE'].iloc[i] += df1519[filt]['WHITE'] * dec
            neigh1519['AFRICAN_AMERICAN'].iloc[i] += df1519[filt]['AFRICAN_AMERICAN'] * dec
            neigh1519['NATIVE_AMERICAN'].iloc[i] += df1519[filt]['NATIVE_AMERICAN'] * dec
            neigh1519['ASIAN'].iloc[i] += df1519[filt]['ASIAN'] * dec
            neigh1519['OTHER'].iloc[i] += df1519[filt]['OTHER'] * dec

for c in neigh1519.columns:
    if c != 'NEIGHBORHOOD':
        neigh1519[c] = neigh1519[c].round().astype(int)

cols = list(neigh1519.columns)[2:]

for c in cols:
    for i in range(len(neigh1519)):
        neigh1519.loc[i, c] = round(neigh1519.loc[i, c]/neigh1519.loc[i, 'TOTAL'], 2)

neigh1519

/Users/yagevlevi/opt/anaconda3/lib/python3.8/site-packages/pandas/core/indexing.py:1637: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_block(indexer, value, name)


,NEIGHBORHOOD,TOTAL,WHITE,AFRICAN_AMERICAN,NATIVE_AMERICAN,ASIAN,OTHER
0,Roslindale,30207,0.61,0.24,0.00,0.02,0.07
1,Jamaica Plain,43871,0.62,0.18,0.00,0.05,0.10
2,Mission Hill,14251,0.51,0.19,0.00,0.18,0.09
3,Longwood,6549,0.60,0.12,0.00,0.20,0.05
4,Bay Village,1465,0.64,0.03,0.00,0.26,0.03
5,Leather District,621,0.48,0.05,0.00,0.43,0.01
6,Chinatown,3113,0.50,0.05,0.00,0.41,0.01
7,North End,6785,0.91,0.02,0.00,0.04,0.02
8,Roxbury,53772,0.20,0.54,0.01,0.04,0.15
9,South End,24202,0.62,0.11,0.00,0.16,0.06


In [89]:
neigh1519.to_csv('Boston_Neighborhood_Demographics_Based_On_ACS_5Year_Estimates_2015-19.csv')

In [51]:
data2011 = pd.concat([adjusted11, neigh1015], axis=1)
data2011 = data2011.drop(25)
data2013 = pd.concat([adjusted13, neigh1015], axis=1)
data2013 = data2013.drop(25)
data2015 = pd.concat([adjusted15, neigh1015], axis=1)
data2015 = data2015.drop(25)
data2017 = pd.concat([adjusted17, neigh1519], axis=1)
data2017 = data2017.drop(25)